<a href="https://colab.research.google.com/github/JacoDuToit11/ComputerVisionProjects/blob/main/PetEmotionClassification/PetEmotionClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
happy_folder = "/content/drive/MyDrive/PetEmotionData/happy"
sad_folder = "/content/drive/MyDrive/PetEmotionData/Sad"
angry_folder = "/content/drive/MyDrive/PetEmotionData/Angry"

# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (48, 48))  # Resize to a fixed size for the model
            images.append(img)
    return images

# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)

happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)

# Concatenate images and labels
X = np.array(happy_images + sad_images + angry_images)
y = np.array(happy_labels + sad_labels + angry_labels)

# Normalize pixel values to range [0, 1]
X = X.astype('float32') / 255.0

# One-hot encode the labels
one_hot_y = np.eye(np.max(y) + 1)[y]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, test_size=0.2, random_state=42)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.y[index]

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    # Add more transformations if needed (e.g., normalization)
])

train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
# Basic CNN Classifier
class CNNClassifier(nn.Module):
  def __init__(self, num_classes):
    super(CNNClassifier, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
    self.relu = nn.ReLU()
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
    self.fc1 = nn.Linear(32 * 12 * 12, 256)
    self.fc2 = nn.Linear(256, num_classes)

  def forward(self, x):
    x = self.conv1(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)
    x = x.view(-1, 32 * 12 * 12)
    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)
    return x

num_classes = len(y_train[0])  # Assuming y_train is one-hot encoded
model = CNNClassifier(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, torch.argmax(labels, dim=1))  # Assuming labels are one-hot encoded
        loss.backward()
        optimizer.step()

    # Print training loss for each epoch
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Evaluation on the test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20, Loss: 1.096474528312683
Epoch 2/20, Loss: 1.0862946510314941
Epoch 3/20, Loss: 1.0836280584335327
Epoch 4/20, Loss: 1.165968418121338
Epoch 5/20, Loss: 0.9758689403533936
Epoch 6/20, Loss: 0.9500083327293396
Epoch 7/20, Loss: 0.740239143371582
Epoch 8/20, Loss: 0.772416353225708
Epoch 9/20, Loss: 0.8514420986175537
Epoch 10/20, Loss: 0.3979153335094452
Epoch 11/20, Loss: 0.5576593279838562
Epoch 12/20, Loss: 0.3101760447025299
Epoch 13/20, Loss: 0.3495209217071533
Epoch 14/20, Loss: 0.17198903858661652
Epoch 15/20, Loss: 0.17528913915157318
Epoch 16/20, Loss: 0.10109645128250122
Epoch 17/20, Loss: 0.13967536389827728
Epoch 18/20, Loss: 0.11909573525190353
Epoch 19/20, Loss: 0.07473278790712357
Epoch 20/20, Loss: 0.09218872338533401
Test Accuracy: 52.67%


In [27]:
happy_folder = "/content/drive/MyDrive/PetEmotionData/happy"
sad_folder = "/content/drive/MyDrive/PetEmotionData/Sad"
angry_folder = "/content/drive/MyDrive/PetEmotionData/Angry"

# Function to load and preprocess images
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))  # Resize to a fixed size for the model
            images.append(img)
    return images

# Load images and labels for each emotion
happy_images = load_images_from_folder(happy_folder)
sad_images = load_images_from_folder(sad_folder)
angry_images = load_images_from_folder(angry_folder)

happy_labels = [0] * len(happy_images)
sad_labels = [1] * len(sad_images)
angry_labels = [2] * len(angry_images)

# Concatenate images and labels
X = np.array(happy_images + sad_images + angry_images)
y = np.array(happy_labels + sad_labels + angry_labels)

# Normalize pixel values to range [0, 1]
X = X.astype('float32') / 255.0

# One-hot encode the labels
one_hot_y = np.eye(np.max(y) + 1)[y]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, test_size=0.2, random_state=42)

In [28]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        label = self.y[index]

        # Apply transformations if provided
        if self.transform:
            image = self.transform(image)

        return image, label

transform = transforms.Compose([
    transforms.ToTensor(),  # Convert image to PyTorch tensor
    # Add more transformations if needed (e.g., normalization)
])

train_dataset = CustomDataset(X_train, y_train, transform=transform)
test_dataset = CustomDataset(X_test, y_test, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

In [29]:
# Fine-tune pre-trained CNN model.
class FineTunedResNet18(nn.Module):
    def __init__(self, num_classes):
        super(FineTunedResNet18, self).__init__()
        # Load pre-trained ResNet18 model
        resnet = resnet18(pretrained=True)

        # Remove the last fully connected layer and replace it with a new one
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        self.fc = nn.Linear(resnet.fc.in_features, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

num_classes = len(y_train[0])
model = FineTunedResNet18(num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, torch.argmax(labels, dim=1))  # Assuming labels are one-hot encoded
        loss.backward()
        optimizer.step()

    # Print training loss for each epoch
    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}')

# Evaluation on the test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

accuracy = correct / total
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/20, Loss: 0.8689529299736023
Epoch 2/20, Loss: 0.5605757832527161
Epoch 3/20, Loss: 0.36467456817626953
Epoch 4/20, Loss: 0.6091190576553345
Epoch 5/20, Loss: 0.17594243586063385
Epoch 6/20, Loss: 0.08836078643798828
Epoch 7/20, Loss: 0.09981609135866165
Epoch 8/20, Loss: 0.14092124998569489
Epoch 9/20, Loss: 0.01220651250332594
Epoch 10/20, Loss: 0.014042079448699951
Epoch 11/20, Loss: 0.14129690825939178
Epoch 12/20, Loss: 0.18214060366153717
Epoch 13/20, Loss: 0.1035454198718071
Epoch 14/20, Loss: 0.029728613793849945
Epoch 15/20, Loss: 0.16873453557491302
Epoch 16/20, Loss: 0.09084930270910263
Epoch 17/20, Loss: 0.06927654147148132
Epoch 18/20, Loss: 0.07205302268266678
Epoch 19/20, Loss: 0.015043962746858597
Epoch 20/20, Loss: 0.11300741881132126
Test Accuracy: 78.67%
